In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
import src.config as config
from src.inference import (
    get_feature_store
)

from datetime import datetime, timezone, timedelta

def get_current_utc_hour_ceiled():
    """Get current UTC time ceiled to the next hour"""
    current = datetime.now(timezone.utc)
    if current.minute > 0 or current.second > 0:
        # Ceil to next hour
        current = current + timedelta(hours=1)

    # Reset minutes, seconds, and microseconds
    current = current.replace(minute=0, second=0, microsecond=0)
    return current

# Usage
current_hour = get_current_utc_hour_ceiled()
print(f"Current UTC hour (ceiled): {current_hour}")

Current UTC hour (ceiled): 2025-03-06 09:00:00+00:00


In [4]:
def fetch_predictions():
    current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour == current_hour)

    return query.read()

In [5]:
fs = get_feature_store()
fg = fs.get_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1
)

query = fg.select_all()

2025-03-06 03:03:52,747 INFO: Initializing external client
2025-03-06 03:03:52,747 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 03:03:53,966 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215672


In [6]:
df= query.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.60s) 


In [7]:
df

,pickup_location_id,predicted_demand,pickup_hour
0,150,0.0,2025-03-04 00:00:00+00:00
1,131,0.0,2025-03-04 00:00:00+00:00
2,74,8.0,2025-03-04 00:00:00+00:00
3,253,0.0,2025-03-04 00:00:00+00:00
4,201,0.0,2025-03-04 00:00:00+00:00
...,...,...,...
10556,21,0.0,2025-03-06 08:00:00+00:00
10557,199,0.0,2025-03-06 08:00:00+00:00
10558,249,53.0,2025-03-06 08:00:00+00:00
10559,215,1.0,2025-03-06 08:00:00+00:00


In [8]:
from datetime import datetime, timezone, timedelta

def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    # Extract components from the rounded-up time
    year = next_hour.year
    month = next_hour.month
    day = next_hour.day
    hour = next_hour.hour

    # Create date strings in YYYY-MM-DD format
    current_date = f"{year}-{month:02d}-{day:02d}"
    next_date = next_hour + timedelta(days=1)
    next_date_str = next_date.strftime("%Y-%m-%d")

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    # # First get the full day's data from Hopsworks
    # df = fg.filter(
    #     (fg.pickup_hour >= current_date) &
    #     (fg.pickup_hour < next_date_str)
    # ).read()

    # Then filter for next hour in the DataFrame
    hour_str = f"{current_date} {hour:02d}:00"
    df_hour = df[df['pickup_hour'] == hour_str]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Querying for date range: {current_date} to {next_date_str}")
    print(f"Filtering for hour: {hour_str}")
    print(f"Found {len(df_hour)} records")
    return df_hour

# Example usage:
predictions = fetch_next_hour_predictions()

2025-03-06 03:03:57,030 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 03:03:57,046 INFO: Initializing external client
2025-03-06 03:03:57,046 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 03:03:58,045 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215672
Current UTC time: 2025-03-06 08:03:57.030577+00:00
Next hour: 2025-03-06 09:00:00+00:00
Querying for date range: 2025-03-06 to 2025-03-07
Filtering for hour: 2025-03-06 09:00
Found 0 records


In [9]:
now = datetime.now(timezone.utc)

In [10]:
predictions

,pickup_location_id,predicted_demand,pickup_hour


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10561 entries, 0 to 10560
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_location_id  10561 non-null  int32                  
 1   predicted_demand    10561 non-null  float64                
 2   pickup_hour         10561 non-null  datetime64[us, Etc/UTC]
dtypes: datetime64[us, Etc/UTC](1), float64(1), int32(1)
memory usage: 206.4 KB


In [12]:
import pandas as pd
current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

In [13]:
dt = current_date.ceil('h')

In [14]:
fs = get_feature_store()  
fg = fs.get_feature_group(  
    name=config.FEATURE_GROUP_MODEL_PREDICTION,  
    version=1  
)  
query = fg.select_all() 
# query = query.filter(fg.pickup_hour >= "2025-02-13 04:00")
query = query.filter(fg.pickup_hour > "2025-02-13 05:00:00")  
results = query.read()

2025-03-06 03:03:59,427 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 03:03:59,440 INFO: Initializing external client
2025-03-06 03:03:59,440 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 03:04:00,442 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215672
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.92s) 


In [15]:
fg.filter((fg.pickup_hour >= "2025-02-13") & (fg.pickup_hour < "2025-02-14")).read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.46s) 


,pickup_location_id,predicted_demand,pickup_hour


In [16]:
results

,pickup_location_id,predicted_demand,pickup_hour
0,150,0.0,2025-03-04 00:00:00+00:00
1,131,0.0,2025-03-04 00:00:00+00:00
2,74,8.0,2025-03-04 00:00:00+00:00
3,253,0.0,2025-03-04 00:00:00+00:00
4,201,0.0,2025-03-04 00:00:00+00:00
...,...,...,...
10556,21,0.0,2025-03-06 08:00:00+00:00
10557,199,0.0,2025-03-06 08:00:00+00:00
10558,249,53.0,2025-03-06 08:00:00+00:00
10559,215,1.0,2025-03-06 08:00:00+00:00


In [17]:
def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    fs = get_feature_store()
    fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)

    # Then filter for next hour in the DataFrame
    df_hour = df[df["pickup_hour"] == next_hour]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Found {len(df_hour)} records")
    return df_hour

In [18]:
predictions = fetch_next_hour_predictions()

2025-03-06 03:04:04,014 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 03:04:04,024 INFO: Initializing external client
2025-03-06 03:04:04,027 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 03:04:04,967 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215672
Current UTC time: 2025-03-06 08:04:04.014712+00:00
Next hour: 2025-03-06 09:00:00+00:00
Found 0 records


In [22]:
fs = get_feature_store()
fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)

2025-03-06 03:15:02,015 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 03:15:02,031 INFO: Initializing external client
2025-03-06 03:15:02,032 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 03:15:03,212 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215672


In [23]:
df = fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.54s) 


In [24]:
df["pickup_hour"].max()

Timestamp('2025-03-06 08:00:00+0000', tz='Etc/UTC')

In [25]:
now = datetime.now(timezone.utc)
next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
print(next_hour)

2025-03-06 09:00:00+00:00


In [26]:
df[df["pickup_hour"] == next_hour]

,pickup_location_id,predicted_demand,pickup_hour
